# Scala BNH

In [ ]:
interp.load.ivy("ai.quantnet" %% "qnt-scala" % System.getenv("QNT_SCALA_VERSION"))

In [ ]:
import ai.{quantnet => qn }
import java.time.LocalDate

In [ ]:
val dateFrom = LocalDate.of(2015, 1, 1)
val dateTo = LocalDate.now

In [ ]:
val assetInfos = qn.data.loadStockList(dateFrom, dateTo)

In [ ]:
val series = qn.data.loadStockDailySeries(assetInfos.map(_.id), dateFrom, dateTo)

In [ ]:
val liquid = series(qn.data.fields.is_liquid)

In [ ]:
liquid.toString()

In [ ]:
val output = liquid.fillLike(0d)
for(ri <- liquid.rowIdx.indices) {
    var sum = 0d
    for(ci <- liquid.colIdx.indices) {
        val v = liquid.data(ri, ci)
        if(v.isFinite) {
            sum += math.abs(v)
        }
    }
    if(sum < 1) {
        sum = 1
    }
    for(ci <- liquid.colIdx.indices) {
        output.data(ri, ci) = liquid.data(ri, ci) / sum
    }
}
output.toString

In [ ]:
val stat = qn.stats.calcStats(series, output)
stat.toString()

In [ ]:
val rr = stat.colOps.get(qn.stats.fields.relativeReturn)

In [ ]:
qn.stats.calcCorrelation(rr)

In [ ]:
qn.data.writeOutput(output)

In [ ]:
interp.load.ivy("org.plotly-scala" %% "plotly-almond" % System.getenv("PLOTLY_ALMOND_VERSION"))
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [ ]:
val sharpe = stat.colOps.get(qn.stats.fields.sharpeRatio)

In [ ]:
val trace1 = Scatter(
  sharpe.idx.map(i=>i.toString).toSeq,
  sharpe.data.toArray.toSeq
)

val data = Seq(trace1)

plot(data)